# This notebook produces 

 1. Box plots of SSA, density and anisotropy for ASMEx slabs (fig 2)
 2. Scatter plot of ASMEx absorption coefficients (fig 7)
 3. Scatter plot of ASMEx scattering coefficients (fig 8)
 4. Brightness temperature of slabs for ASMEx data (fig 6a)

In [ ]:
# Import statements
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
from matplotlib.ticker import LogLocator, NullFormatter


# local import
from smrt import make_snowpack, make_emmodel, sensor_list, make_model, make_atmosphere
from smrt.substrate.reflector import make_reflector
from smrt.core.globalconstants import DENSITY_OF_ICE, FREEZING_POINT, C_SPEED
from common_functions import symmetrize_microstructure, rmse, me, regression_coefficient, microstructure_colour_list
from common_functions import microstructure_short_labels, microstructure_symbols

### Useful functions

In [ ]:
# Functions for plot characteristics
def slab_symbol_list():
    return ['o', 'v', '^', '<', '>', 's', 'p', 'P', '*', 'h', 'H', 'X', 'D', 'd']

def frequency_colour_list():
    return {18.7e9: 'c', 21e9:'b', 36.5e9:'r', 90e9:'k'}

def set_frequency_in_Hz(df):
    # Converts GHz to Hz to be consistent with simulations and corrects 89GHz naming error to 90GHz

    freq = list(df.frequency.values)
    # Find where frequency is in list
    fil = freq.index(89)
    # Replace 89 with 90
    freq[fil] = 90
    # Rewrite coordinates in Hz
    df['frequency'] = np.array(freq)*1e9
    return df

def wavenumber(f):
    return 2 * np.pi * f / C_SPEED

### Read in field data

#### Microstructure

In [ ]:
# Read microstructure
# Get list of files
microstructure_files = [os.path.join(root, name)
             for root, dirs, files in os.walk('data/ASMEx/')
             for name in files
             if name.endswith(("EMMS_acf_parameters_v1.0.csv"))]

# Create dataframe of microstructure properties per subsample, assuming spherical symmetry
df = pd.concat((symmetrize_microstructure(f) for f in microstructure_files), ignore_index=True)

#### Temperature

In [ ]:
# Read ASMEx temperature and create dataframe
# This file has no headers, so they must be defined
slab_list = ['A01','A02','A03','A04','A05','A06','A07','B01','B02','B03','B04','B05','B06','B07']
#Rows 1, 2, and 3 are the temperatures on side 1, and rows 4, 5, and 6 are the temperatures on side 2.
#Rows 1 and 4 are temperatures at 15cm from the base,
#Rows 2 and 5 are temperatures at 10 cm from the base,
#Rows 3 and 6 are temperatures at 5 cm from the base.
dft = pd.read_csv('data/ASMEx/asmex_temps.csv', names=slab_list) + FREEZING_POINT

#### Scattering and absorption coefficients

In [ ]:
# Read ASMEx observed scattering coefficients
## Reads in .xlsx, sets index to be slab and frequency
## Then converts to xarray and removes unused data
Vobs = pd.read_excel('data/ASMEx/200504_ASMEx_FCM_values.xlsx', sheet_name='V-Pol').set_index(['Sample', 'f (ghz)']).to_xarray().drop(labels=['eabs', 'emet', 
                                                                                                           'r0', 't0', 'ya\'', 'ys\''])
Vobs = set_frequency_in_Hz(Vobs.rename({'Sample':'slab', 'f (ghz)':'frequency'}))
Hobs = pd.read_excel('data/ASMEx/200504_ASMEx_FCM_values.xlsx', sheet_name='H-Pol').set_index(['Sample', 'f (ghz)']).to_xarray().drop(labels=['eabs', 'emet', 
                                                                                                           'r0', 't0', 'ya\'', 'ys\''])
Hobs = set_frequency_in_Hz(Hobs.rename({'Sample':'slab', 'f (ghz)':'frequency'}))

#### Brightness temperature

In [ ]:
# Read ASMEx TB
obs_tb = pd.read_csv('data/ASMEx/asmex_tb.csv').dropna().set_index(['Sample', 'Freq']).to_xarray().rename({'Sample':'slab', 'Freq':'frequency'})
# Change units to be consistent with simulations
obs_tb = set_frequency_in_Hz(obs_tb)

#### Slab thickness

In [ ]:
# Read ASMEx depth
obs_depth = pd.read_csv('data/ASMEx/asmex_depths.csv', encoding='raw_unicode_escape').set_index('Slab Reference').dropna()

#### Slab emissivity

In [ ]:
# Read ASMEx Emissivities
emiss = pd.read_csv('data/ASMEx/asmex_emissivities.csv').set_index('Slab').T.drop('Design')
# Add in NaN's for A04: no measurements available
emiss.insert(3, 'A04', [np.nan]*len(emiss))

# Deal with missing data - replace with mean of all available observations at that freq / pol
emiss = emiss.T.fillna(np.round(emiss.mean(axis=1),2)).T

# Extract frequency and polarization from index
emiss['frequency'] = emiss.index.str[:-1]
emiss['polarization'] = emiss.index.str[-1]

# Rename 89 GHz to 90 GHz - naming problem in original file
emiss.replace(to_replace='89', value='90', inplace=True)
emiss.replace(to_replace='19', value='18.7', inplace=True)
emiss.replace(to_replace='37', value='36.5', inplace=True)

# Put into xarray format
emiss = emiss.set_index(['frequency', 'polarization']).to_xarray()

# Change frequency to GHz
emiss['frequency'] = [(float(m)*1e9) for m in emiss.frequency.values]

### Model configuration

In [ ]:
# ASMEx radiometers (21GHz is most complete dataset)
rad19 = sensor_list.passive(18.7e9, 50)
rad21 = sensor_list.passive(21e9, 50)
rad37 = sensor_list.passive(36.5e9, 50)
rad90 = sensor_list.passive(90e9, 50)
rads = sensor_list.passive([18.7e9, 21e9, 36.5e9, 90e9, 150e9], 50)

In [ ]:
# Run SMRT to get TB
model = make_model('iba', 'dort', rtsolver_options=dict(error_handling='nan'))

## 1. Figure 2 - box plots of SSA, density and anisotropy

In [ ]:
# Need to read in microstructure data again, but without spherical symmetry assumption
df = pd.concat((pd.read_csv(f) for f in microstructure_files), ignore_index=True)
# Extract file id and add to dataframe
df['fileid'] = df[['1:filename']]
df['id'] = df.fileid.str[43:47]
# Calculate anisotropy factor from correlation lengths and add to dataframe
df['aniso'] = 2.0 * df[' 5:l_c_z'] / (df[' 3:l_c_x'] + df[' 4:l_c_y'])

# Plot graph
flierprops = dict(marker='o', markerfacecolor='black', markersize=2, linestyle='none')
plt.close()
plt.rcParams.update({'font.size': 16})
f, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, sharey=False, figsize=(10,8))
df.boxplot(column=[' 2:phi_2'], by=['id'], grid=False, rot=90, flierprops=flierprops, ax=ax1)
df.boxplot(column=[' 5:l_c_z'], by=['id'], grid=False, rot=90, flierprops=flierprops, ax=ax2)
df.boxplot(column=['aniso'], by=['id'], grid=False, rot=90, flierprops=flierprops, ax=ax3)
ax1.set_title('')
ax2.set_title('')
ax3.set_title('')
ax1.set_ylabel('$\phi$')
ax2.set_ylabel('$l_z$ (mm)')
ax3.set_ylabel('$2l_z/(l_x+l_y)$')
ax3.set_xlabel('Slab Sample')
ax3.plot(np.ones(len(np.unique(df.id))+2),'r--')
ax1.yaxis.set_major_locator(plt.MaxNLocator(6))
ax2.yaxis.set_major_locator(plt.MaxNLocator(5))
ax3.yaxis.set_major_locator(plt.MaxNLocator(5))
ax3.set_xlim(0,len(np.unique(df.id))+1)
plt.suptitle('')

# Seasonal difference
ax1.axvline(x=1+len(np.unique(df.id))/2,color='lightgray')
ax2.axvline(x=1+len(np.unique(df.id))/2,color='lightgray')
ax3.axvline(x=1+len(np.unique(df.id))/2,color='lightgray')

# Fine-tune figure; make subplots close to each other and hide x ticks for
# all but bottom plot.
f.subplots_adjust(hspace=0)
plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)

## 2. Absorption coefficient

#### Useful functions

In [ ]:
# Calculates absorption coefficient, given SMRT snowpack layers
def calc_ka(sp):
    # Collate all ka for a given snowpack (sp)
    ka19 = [make_emmodel('iba', rad19, sp.layers[l]).ka for l in range(len(sp.layers))]
    ka21 = [make_emmodel('iba', rad21, sp.layers[l]).ka for l in range(len(sp.layers))]
    ka37 = [make_emmodel('iba', rad37, sp.layers[l]).ka for l in range(len(sp.layers))]
    ka90 = [make_emmodel('iba', rad90, sp.layers[l]).ka for l in range(len(sp.layers))]
    ka = xr.DataArray([ka19, ka21, ka37, ka90], coords=[('frequency', [18.7e9, 21e9, 36.5e9, 90e9]), 
            ('layer', range(len(sp.layers)))]).assign_coords(microstructure=sp.layers[0].microstructure_model.__name__)
    return ka

# Generates snowpack layers, applies function to calculate absorption coefficient
def slab_ka(df, temperature, slab):
    # Returns all ka for a particular slab
    # Make snowpack for each microstructure
    sne = calc_ka(make_snowpack(thickness=np.ones(len(df)), temperature=temperature, density=df.density, 
                        microstructure_model="exponential", corr_length=df.l_ex))
    sni = calc_ka(make_snowpack(thickness=np.ones(len(df)), temperature=temperature, density=df.density, 
                        microstructure_model="independent_sphere", radius=df.d_sph/2))
    sns = calc_ka(make_snowpack(thickness=np.ones(len(df)), temperature=temperature, density=df.density, 
                        microstructure_model="sticky_hard_spheres", radius=df.d_shs/2, stickiness=df.tau))
    snt = calc_ka(make_snowpack(thickness=np.ones(len(df)), temperature=temperature, density=df.density, 
                        microstructure_model="teubner_strey", corr_length=df.xi_ts, repeat_distance=df.domain_ts))
    sng = calc_ka(make_snowpack(thickness=np.ones(len(df)), temperature=temperature, density=df.density, 
                        microstructure_model="gaussian_random_field", corr_length=df.xi_grf, repeat_distance=df.domain_grf))
    return xr.concat([sne, sni, sns, snt, sng], dim='microstructure').assign_coords(slab=slab)

# Plots obs vs sims for a particular frequency and microstructure model
def slab_scatter_plot(frequency, microstructure, Hobs, Vobs, sims):
    # Scale by 1/k0
    scale = 1 / wavenumber(frequency)
    # Calculate mean of Hobs and Vobs
    obs_mean = (Hobs.sel(frequency=frequency) + Vobs.sel(frequency=frequency)) / 2
    sim_mean = sims.sel(microstructure=microstructure, frequency=frequency).mean(dim='layer')
    # Calculate y error bars (x error bars calculated by slab in plt.errorbar call below)
    upper_yerr = sims.sel(microstructure=microstructure, frequency=frequency).max(dim='layer') - sim_mean
    lower_yerr = sim_mean - sims.sel(microstructure=microstructure, frequency=frequency).min(dim='layer')

    [ax.errorbar(scale * obs_mean.sel(slab=sl).ya, scale * sim_mean.sel(slab=sl), 
                  yerr=[[scale * lower_yerr.sel(slab=sl)], [scale * upper_yerr.sel(slab=sl)]], # lower then upper
                  xerr=[[scale * (obs_mean.sel(slab=sl).ya - min(Vobs.ya.sel(frequency=frequency, slab=sl), Hobs.ya.sel(frequency=frequency, slab=sl)))], 
                        [scale * (max(Vobs.ya.sel(frequency=frequency, slab=sl), Hobs.ya.sel(frequency=frequency, slab=sl)) - obs_mean.sel(slab=sl).ya)]],
                  marker=slab_symbol_list()[i], c=frequency_colour_list()[float(frequency)], markersize=14,
                alpha=0.5) for i, sl in enumerate(slab_list)]
    
def slab_scatter_plot_noscale(frequency, microstructure, Hobs, Vobs, sims):

    plt.yscale('log')
    plt.xscale('log')
    # Calculate mean of Hobs and Vobs
    obs_mean = (Hobs.sel(frequency=frequency) + Vobs.sel(frequency=frequency)) / 2
    sim_mean = sims.sel(microstructure=microstructure, frequency=frequency).mean(dim='layer')
    # Calculate y error bars (x error bars calculated by slab in plt.errorbar call below)
    upper_yerr = sims.sel(microstructure=microstructure, frequency=frequency).max(dim='layer') - sim_mean
    lower_yerr = sim_mean - sims.sel(microstructure=microstructure, frequency=frequency).min(dim='layer')

    [ax.errorbar(obs_mean.sel(slab=sl).ya, sim_mean.sel(slab=sl), 
                  yerr=[[lower_yerr.sel(slab=sl)], [upper_yerr.sel(slab=sl)]], # lower then upper
                  xerr=[[(obs_mean.sel(slab=sl).ya - min(Vobs.ya.sel(frequency=frequency, slab=sl), Hobs.ya.sel(frequency=frequency, slab=sl)))], 
                        [(max(Vobs.ya.sel(frequency=frequency, slab=sl), Hobs.ya.sel(frequency=frequency, slab=sl)) - obs_mean.sel(slab=sl).ya)]],
                  marker=slab_symbol_list()[i], c=frequency_colour_list()[float(frequency)], markersize=14,
                alpha=0.5) for i, sl in enumerate(slab_list)]

#### Calculation of absorption coefficients

In [ ]:
ka_list = [] # list of all ka per slab: to be combined in xarray
for sl in slab_list: 
    # Get observed slab temperature
    temperature = dft.mean()[sl]
    # Get list of files that relate to slab
    slab_files = [f for f in microstructure_files if sl in f]
    # Make microstructure dataframe out of list
    df = pd.concat((symmetrize_microstructure(f) for f in slab_files), ignore_index=True)
    # Add slab ka to list
    ka_list.append(slab_ka(df, temperature, sl))
all_ka = xr.concat(ka_list, dim='slab') # NB will be NaNs in here as not all slabs have same number of layers

#### Plot Figure 7

In [ ]:
plt.close()
plt.rcParams.update({'font.size': 16})
fig, ax = plt.subplots(figsize=(10, 10))

frequency_list = all_ka.frequency.values

# Plot ka for each frequency
[slab_scatter_plot(f, 'Exponential', Hobs, Vobs, all_ka) for f in frequency_list]


# 1:1 line
xmin = 0
xmax = 0.002
ax.plot([xmin, xmax], [xmin, xmax], 'b--', alpha=0.2)
plt.xlim(xmin,xmax)
plt.ylim(xmin,xmax)
    
# Custom legend
# Slab
legend_elements1 = [Line2D([0], [0], marker=slab_symbol_list()[i], color='w', label=sl,
                          markerfacecolor='grey', markersize=12) for i, sl in enumerate(slab_list)]
legend1 = ax.legend(handles=legend_elements1, loc='upper left', ncol=2, title='Slab (symbol)')
# Frequency
legend_elements2 = [Line2D([0], [0], marker='s', color='w', label=f/1e9,
                          markerfacecolor=frequency_colour_list()[f], markersize=12) for f in frequency_list]
legend2 = ax.legend(handles=legend_elements2, loc='lower right', title='Frequency [GHz]', ncol=2)

ax.add_artist(legend1)
ax.add_artist(legend2)

ax.set_xlabel('Observed $\kappa_a$ / $k_0$ []')
ax.set_ylabel('Simulated $\kappa_a$ / $k_0$ []')
ax.set_xticks(np.arange(0, 0.0025, 0.0005))
ax.set_yticks(np.arange(0, 0.0025, 0.0005))

plt.savefig('Fig7.png')

#### Calculate error statistics

In [ ]:
# Calc RMSE, mean error
obs_mean = (Hobs + Vobs).ya / 2
sim_mean = all_ka.sel(microstructure='Exponential').mean(dim='layer')

In [ ]:
print (rmse(obs_mean, sim_mean))
print (me(obs_mean, sim_mean))
print (regression_coefficient(obs_mean.isel(frequency=slice(0,4)), sim_mean))

## 3. Scattering coefficient

#### Useful functions

In [ ]:
# Calculates scattering coefficient, given SMRT snowpack layers
def calc_ks(sp):
    # Collate all ka for a given snowpack (sp)
    ks19 = [make_emmodel('iba', rad19, sp.layers[l]).ks for l in range(len(sp.layers))]
    ks21 = [make_emmodel('iba', rad21, sp.layers[l]).ks for l in range(len(sp.layers))]
    ks37 = [make_emmodel('iba', rad37, sp.layers[l]).ks for l in range(len(sp.layers))]
    ks90 = [make_emmodel('iba', rad90, sp.layers[l]).ks for l in range(len(sp.layers))]
    ks = xr.DataArray([ks19, ks21, ks37, ks90], coords=[('frequency', [18.7e9, 21e9, 36.5e9, 90e9]), 
            ('layer', range(len(sp.layers)))]).assign_coords(microstructure=sp.layers[0].microstructure_model.__name__)
    return ks

# Generates snowpack layers, applies function to calculate absorption coefficient
def slab_ks(df, temperature, slab):
    # Returns all ka for a particular slab
    # Make snowpack for each microstructure
    sne = calc_ks(make_snowpack(thickness=np.ones(len(df)), temperature=temperature, density=df.density, 
                        microstructure_model="exponential", corr_length=df.l_ex))
    sni = calc_ks(make_snowpack(thickness=np.ones(len(df)), temperature=temperature, density=df.density, 
                        microstructure_model="independent_sphere", radius=df.d_sph/2))
    sns = calc_ks(make_snowpack(thickness=np.ones(len(df)), temperature=temperature, density=df.density, 
                        microstructure_model="sticky_hard_spheres", radius=df.d_shs/2, stickiness=df.tau))
    snt = calc_ks(make_snowpack(thickness=np.ones(len(df)), temperature=temperature, density=df.density, 
                        microstructure_model="teubner_strey", corr_length=df.xi_ts, repeat_distance=df.domain_ts))
    sng = calc_ks(make_snowpack(thickness=np.ones(len(df)), temperature=temperature, density=df.density, 
                        microstructure_model="gaussian_random_field", corr_length=df.xi_grf, repeat_distance=df.domain_grf))
    return xr.concat([sne, sni, sns, snt, sng], dim='microstructure').assign_coords(slab=slab)



# Plots scattering coefficients vs retrieved value for slab
def ks_obs_scatter_plot_noscale(microstructure, hide_ax=False):
    # Plots obs vs sims for a particular frequency and microstructure model
    
    plt.yscale('log')
    plt.xscale('log')
    xmin = 1e-4
    xmax = 1e3 # Unscaled
    for f in all_ks.coords['frequency'].sortby(all_ks.coords['frequency'], ascending=False):


        # Calculate mean of Hobs and Vobs
        obs_mean = (Hobs.sel(frequency=f) + Vobs.sel(frequency=f)) / 2
        sim_mean = all_ks.sel(microstructure=microstructure, frequency=f).mean(dim='layer')
        # Calculate y error bars (x error bars calculated by slab in plt.errorbar call below)
        upper_yerr = all_ks.sel(microstructure=microstructure, frequency=f).max(dim='layer') - sim_mean
        lower_yerr = sim_mean - all_ks.sel(microstructure=microstructure, frequency=f).min(dim='layer')

        [plt.errorbar(obs_mean.sel(slab=sl).ys, sim_mean.sel(slab=sl), 
                      yerr=[[lower_yerr.sel(slab=sl)], [upper_yerr.sel(slab=sl)]], # lower then upper
                      xerr=[[(obs_mean.sel(slab=sl).ys - min(Vobs.ys.sel(frequency=f, slab=sl), Hobs.ys.sel(frequency=f, slab=sl)))], 
                            [(max(Vobs.ys.sel(frequency=f, slab=sl), Hobs.ys.sel(frequency=f, slab=sl)) - obs_mean.sel(slab=sl).ys)]],
                          marker='.', ls='--', linewidth=0.5, alpha=0.5, markersize=6,
                          c=frequency_colour_list()[float(f)]) for i, sl in enumerate(slab_list)]
    
    plt.plot([xmin,xmax], [xmin,xmax], 'k--', alpha=0.2)
    plt.xlim(xmin,xmax)
    plt.ylim(xmin,xmax)
    # Calc best fit line
    obs = (Hobs.ys + Vobs.ys)/2
    # Exclude 150GHz obs
    x = obs.isel(frequency=slice(0,4))

    gradient, intercept, rsq_value = regression_coefficient(x, all_ks.sel(microstructure=microstructure).mean(dim='layer'))
    
    # Hide axis labels
    if hide_ax==True:
        plt.xticks([])
    else:
        plt.minorticks_on()
        plt.xlabel('$\overline{\kappa_s}$ [$m^{-1}$]') # Unscaled
    plt.yticks([]) # Hide these for all figures

    txt = 'r$^2$=' + str(np.round(rsq_value,2))
    plt.text(1e-3,1e2,txt) # Unscaled)

# Plots scattering coefficients for one microstructure model vs another
def plot_ks_noscale(micro1, micro2, hide_ax=False):
    
    # log axis
    plt.yscale('log')
    plt.xscale('log')
    xmin = 1e-6 # Log
    xmax = 1e4 # Log

    [plt.scatter(all_ks.sel(microstructure=micro1, frequency=f), all_ks.sel(microstructure=micro2, frequency=f) ,
                marker='.', color=(frequency_colour_list()[float(f)]), alpha=0.5)
         for f in all_ks.coords['frequency'].sortby(all_ks.coords['frequency'], ascending=False)]
    plt.plot([xmin, xmax], [xmin,xmax], 'k--', alpha=0.2)
    plt.xlim(xmin,xmax)
    plt.ylim(xmin,xmax)
    gradient, intercept, rsq_value = regression_coefficient(all_ks.sel(microstructure=micro1), all_ks.sel(microstructure=micro2))

    
    # r^2 value
    txt = 'r$^2$=' + str(np.round(rsq_value,2))

    # Hide axis labels
    if hide_ax==True:
        plt.xticks([])
    else:
        plt.xlabel('$\kappa_s$ [$m^{-1}$]') # Unscaled
    plt.yticks([]) # Hide these for all figures
    plt.text(1e-5, 4e2, txt) # Log 

#### Generate scattering coefficients

In [ ]:
ks_list = [] # list of all ka per slab: to be combined in xarray
for sl in slab_list: 
    # Get observed slab temperature
    temperature = dft.mean()[sl]
    # Get list of files that relate to slab
    slab_files = [f for f in microstructure_files if sl in f]
    # Make microstructure dataframe out of list
    df = pd.concat((symmetrize_microstructure(f) for f in slab_files), ignore_index=True)
    # Add slab ka to list
    ks_list.append(slab_ks(df, temperature, sl))
all_ks = xr.concat(ks_list, dim='slab') # NB will be NaNs in here as not all slabs have same number of layers

#### Plot scattering coefficients - figure 8

In [ ]:
plt.close()
plt.rcParams.update({'font.size': 10})
fig = plt.figure(constrained_layout=False, figsize=(8,8))
spec = fig.add_gridspec(ncols=5, nrows=5, wspace=0.05, hspace=0.05)


# Col 1: OBS
fig.add_subplot(spec[0,0], aspect='equal')
plt.title('OBS', fontsize=14)
ks_obs_scatter_plot_noscale('GaussianRandomField', hide_ax=True)
plt.ylabel('GRF', fontsize=14, rotation='horizontal', labelpad=20)

fig.add_subplot(spec[1,0], aspect='equal')
ks_obs_scatter_plot_noscale('TeubnerStrey', hide_ax=True)
plt.ylabel('TS', fontsize=14, rotation='horizontal', labelpad=20)

fig.add_subplot(spec[2,0], aspect='equal')
ks_obs_scatter_plot_noscale('IndependentSphere', hide_ax=True)
plt.ylabel('IND', fontsize=14, rotation='horizontal', labelpad=20)

fig.add_subplot(spec[3,0], aspect='equal')
ks_obs_scatter_plot_noscale('StickyHardSpheres', hide_ax=True)
plt.ylabel('SHS', fontsize=14, rotation='horizontal', labelpad=20)

fobs = fig.add_subplot(spec[4,0], aspect='equal')
ks_obs_scatter_plot_noscale('Exponential')
plt.ylabel('EXP', fontsize=14, rotation='horizontal', labelpad=20)
# X tick labels for log graph from
# https://stackoverflow.com/questions/44078409/matplotlib-semi-log-plot-minor-tick-marks-are-gone-when-range-is-large
locmaj = LogLocator(base=10,numticks=5) 
fobs.xaxis.set_major_locator(locmaj)
for label in fobs.xaxis.get_ticklabels()[::2]:
    label.set_visible(False)
locmin = LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
fobs.xaxis.set_minor_locator(locmin)
fobs.xaxis.set_minor_formatter(NullFormatter())
#fobs.tick_params(axis='x', which='minor', direction='in')

# Col 2: EXP
fig.add_subplot(spec[0,1], aspect='equal')
plot_ks_noscale('Exponential', 'GaussianRandomField', hide_ax=True)
plt.title('EXP', fontsize=14)

fig.add_subplot(spec[1,1], aspect='equal')
plot_ks_noscale('Exponential', 'TeubnerStrey', hide_ax=True)

fig.add_subplot(spec[2,1], aspect='equal')
plot_ks_noscale('Exponential', 'IndependentSphere', hide_ax=True)


fexp = fig.add_subplot(spec[3,1], aspect='equal')
plot_ks_noscale('Exponential', 'StickyHardSpheres')
# X tick labels for log graph from
locmaje = LogLocator(base=10,numticks=8) 
fexp.xaxis.set_major_locator(locmaje)
for label in fexp.xaxis.get_ticklabels()[1::2]:
    label.set_visible(False)
for label in fexp.xaxis.get_ticklabels()[0::2]:
    label.set_visible(False)
locmine = LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
fexp.xaxis.set_minor_locator(locmine)
fexp.xaxis.set_minor_formatter(NullFormatter())


# Col 3
fig.add_subplot(spec[0,2], aspect='equal')
plot_ks_noscale('StickyHardSpheres', 'GaussianRandomField', hide_ax=True)
plt.title('SHS', fontsize=14)

fig.add_subplot(spec[1,2], aspect='equal')
plot_ks_noscale('StickyHardSpheres', 'TeubnerStrey', hide_ax=True)

fshs = fig.add_subplot(spec[2,2], aspect='equal')
plot_ks_noscale('StickyHardSpheres', 'IndependentSphere')
# X tick labels for log graph from
fshs.xaxis.set_major_locator(locmaje)
for label in fshs.xaxis.get_ticklabels()[1::2]:
    label.set_visible(False)
for label in fshs.xaxis.get_ticklabels()[0::2]:
    label.set_visible(False)
fshs.xaxis.set_minor_locator(locmine)
fshs.xaxis.set_minor_formatter(NullFormatter())

# Col 4
fig.add_subplot(spec[0,3], aspect='equal')
plot_ks_noscale('IndependentSphere', 'GaussianRandomField', hide_ax=True)
plt.title('IND', fontsize=14)

find = fig.add_subplot(spec[1,3], aspect='equal')
plot_ks_noscale('IndependentSphere', 'TeubnerStrey')
# X tick labels for log graph from
find.xaxis.set_major_locator(locmaje)
for label in find.xaxis.get_ticklabels()[1::2]:
    label.set_visible(False)
for label in find.xaxis.get_ticklabels()[0::2]:
    label.set_visible(False)
find.xaxis.set_minor_locator(locmine)
find.xaxis.set_minor_formatter(NullFormatter())

# Put frequency legend here
fig.add_subplot(spec[4,4], aspect='equal')
plt.gca().axis('off')
legend_freq = [Line2D([0], [0], marker='o', color='w',
                      markerfacecolor=frequency_colour_list()[float(f)], label=int(f/1e9)) for f in all_ks.coords['frequency']]
plt.legend(handles=legend_freq, loc='upper left', title='Frequency [GHz]', ncol=1)


# Put slab legend here
#fig.add_subplot(spec[3,4], aspect='equal')
#plt.gca().axis('off')
#legend_slab = [Line2D([0], [0], marker=slab_symbol_list()[i], color='w', label=sl,
#                          markerfacecolor='grey', markersize=12) for i, sl in enumerate(slab_list)]
#plt.legend(handles=legend_slab, loc='center left', ncol=1, title='Slab (symbol)')

# Col 5
fts = fig.add_subplot(spec[0,4], aspect='equal')
plot_ks_noscale('TeubnerStrey', 'GaussianRandomField')
plt.title('TS', fontsize=14)
# X tick labels for log graph from
fts.xaxis.set_major_locator(locmaje)
for label in fts.xaxis.get_ticklabels()[1::2]:
    label.set_visible(False) # Removes every other label
for label in fts.xaxis.get_ticklabels()[0::2]:
    label.set_visible(False) # Then removes middle of remaining labels
fts.xaxis.set_minor_locator(locmine)
fts.xaxis.set_minor_formatter(NullFormatter())


#### Calculate statistics

In [ ]:
# Calculate error statistics
obs_mean = (Hobs + Vobs).ys / 2
sim_mean_exp = all_ks.sel(microstructure='Exponential').mean(dim='layer')
sim_mean_ind = all_ks.sel(microstructure='IndependentSphere').mean(dim='layer')
sim_mean_shs = all_ks.sel(microstructure='StickyHardSpheres').mean(dim='layer')
sim_mean_ts = all_ks.sel(microstructure='TeubnerStrey').mean(dim='layer')
sim_mean_grf = all_ks.sel(microstructure='GaussianRandomField').mean(dim='layer')

In [ ]:
sim_mean = sim_mean_grf
print (rmse(obs_mean, sim_mean))
print (me(obs_mean, sim_mean))
print (regression_coefficient(obs_mean.isel(frequency=slice(0,4)), sim_mean))

## 4. Brightness temperature

#### Useful functions

In [ ]:
# Look up emissivity for each tuple key, calculate reflectivity and store in dictionary
def get_slab_reflectivity(slab):
    refl_dict = {emiss_keys[0] : np.round(1 - emiss[slab].sel(frequency=emiss_keys[0][0], polarization=emiss_keys[0][1]).values,2)}
    for k in emiss_keys[1:]:
        refl_dict.update({k : np.round(1 - emiss[slab].sel(frequency=k[0], polarization=k[1]).values,2)})
    return (refl_dict)

# Create snowpack. 
# snowpack_item is of format 'A01A_abs' meaning slab A01, micro-CT sample A, absorber substrate
def asmex_snowpack(snowpack_item, microstructure):
    slab = snowpack_item[:3]
    sample = snowpack_item[:4]
    substrate = snowpack_item[-3:]
    # Get observed slab temperature
    temperature = dft.mean()[slab]
    # Get list of files that relate to sample
    # Special treatment for A01 (different orientation) and A02A (was not measured)
    if sample == 'A01A':
        # Looking for A01A and A01B
        sample_files = [f for f in microstructure_files if ('A01A' in f) or ('A01B' in f)]
    elif sample == 'A01B':
        # Looking for A01C and A01D
        sample_files = [f for f in microstructure_files if ('A01C' in f) or ('A01D' in f)]
    elif sample == 'A02A':
        sample_files = [f for f in microstructure_files if 'A02' in f]
    else:
        sample_files = [f for f in microstructure_files if sample in f]
    # Need to put Top sample first so can order layers from top to bottom
    sample_files.sort(reverse=True)
    # Make microstructure dataframe out of list
    df = pd.concat((symmetrize_microstructure(f) for f in sample_files), ignore_index=True)
    
    # Calculate layer thickness
    depth_in_m = obs_depth.loc[slab]['Depth (cm)'] * 1e-2
    # Spread evenly between subsample layers
    thickness = [depth_in_m / len(df)] * len(df)
    
    # Assign substrate and atmosphere
    if substrate == 'abs':
        # Absorber - use data from emissivity file
        refl = get_slab_reflectivity(slab)
        reflector = make_reflector(temperature=temperature, specular_reflection=refl)
        # Take mean of H, V pol for atmospheric observations
        tbdown= {f: float(obs_tb.TBAskyH.sel(slab=slab, frequency=f).values + obs_tb.TBAskyV.sel(slab=slab, frequency=f).values) / 2 for f in obs_tb.frequency.values}
        # Replace nan with zero - no observations of sky when no observations of ground
        tbdown = {k: 0 if np.isnan(v) else v for k, v in tbdown.items()}
        atmos = make_atmosphere('simple_isotropic_atmosphere', tbdown=tbdown)
    elif substrate == 'met':
        # Reflector - assume 100% reflecting for now
        reflector = make_reflector(temperature=temperature, specular_reflection=1)
        # Take mean of H, V pol for atmospheric observations
        tbdown= {f: float(obs_tb.TBMskyH.sel(slab=slab, frequency=f).values + obs_tb.TBMskyV.sel(slab=slab, frequency=f).values) / 2 for f in obs_tb.frequency.values}
        # Replace nan with zero - no observations of sky when no observations of ground
        tbdown = {k: 0 if np.isnan(v) else v for k, v in tbdown.items()}
        atmos = make_atmosphere('simple_isotropic_atmosphere', tbdown=tbdown)
    else:
        print ('Substrate ', substrate, ' is not known')   
    
    # Decide which snowpack to return
    if microstructure == 'Exponential':
        return atmos + make_snowpack(thickness=thickness, temperature=temperature, density=df.density, 
                        microstructure_model="exponential", corr_length=df.l_ex, substrate=reflector)
    elif microstructure == 'StickyHardSpheres':
        return atmos + make_snowpack(thickness=thickness, temperature=temperature, density=df.density, 
                        microstructure_model="sticky_hard_spheres", radius=df.d_shs/2, 
                        stickiness=df.tau, substrate=reflector)
    elif microstructure == 'IndependentSphere':
        return atmos + make_snowpack(thickness=thickness, temperature=temperature, density=df.density, 
                        microstructure_model="independent_sphere", radius=df.d_sph/2, substrate=reflector)
    elif microstructure == 'TeubnerStrey':
        return atmos + make_snowpack(thickness=thickness, temperature=temperature, density=df.density, 
                        microstructure_model="teubner_strey", corr_length=df.xi_ts, repeat_distance=df.domain_ts, 
                        substrate=reflector)
    elif microstructure == 'GaussianRandomField':
        return atmos + make_snowpack(thickness=thickness, temperature=temperature, density=df.density, 
                        microstructure_model="gaussian_random_field", corr_length=df.xi_grf, 
                        repeat_distance=df.domain_grf, substrate=reflector)
    else:
        print (microstructure, ' is not a known microstructure model')

# Used in plotting to separate micro-CT sample A and B, and substrate type
def subset_snowpacks(sample_substrate):
    # Create mask
    mask = [sample_substrate in x for x in snowpack_list]
    # Apply it to snowpack list so have one snowpack per slab (14)
    return [snowpack_list[i] for i in range(len(snowpack_list)) if mask[i]]


# Generate plot of simulated vs observed TB, depending on microstructure model
def plot_tb(microstructure):
    # Determine which model results to use
    if microstructure == 'Exponential':
        res = results_exp
    elif microstructure == 'IndependentSphere':
        res = results_ind
    elif microstructure == 'StickyHardSpheres':
        res = results_shs
    elif microstructure == 'TeubnerStrey':
        res = results_ts
    elif microstructure == 'GaussianRandomField':
        res = results_grf
    
    alf = 0.7
    
    # V-pol
    [axv.scatter(obs_tb.TBAV.sel(frequency=f), res.TbV(snowpack=subset_snowpacks('A_abs'), frequency=f), c=microstructure_colour_list()[microstructure], 
               label=microstructure_short_labels()[microstructure], alpha=alf, edgecolors='none', 
                 marker=microstructure_symbols()[microstructure])  for f in obs_tb.frequency] 
    [axv.scatter(obs_tb.TBAV.sel(frequency=f), res.TbV(snowpack=subset_snowpacks('B_abs'), frequency=f), c=microstructure_colour_list()[microstructure], 
               alpha=alf, edgecolors='none', marker=microstructure_symbols()[microstructure]) for f in obs_tb.frequency]
    [axv.scatter(obs_tb.TBMV.sel(frequency=f), res.TbV(snowpack=subset_snowpacks('A_met'), frequency=f), c=microstructure_colour_list()[microstructure],
               label=microstructure_short_labels()[microstructure], alpha=alf, edgecolors='none', 
                 marker=microstructure_symbols()[microstructure])  for f in obs_tb.frequency] 
    [axv.scatter(obs_tb.TBMV.sel(frequency=f), res.TbV(snowpack=subset_snowpacks('B_met'), frequency=f), c=microstructure_colour_list()[microstructure], 
               alpha=alf, edgecolors='none', marker=microstructure_symbols()[microstructure]) for f in obs_tb.frequency]
    
    
    # H-pol
    [axh.scatter(obs_tb.TBAH.sel(frequency=f), res.TbH(snowpack=subset_snowpacks('A_abs'), frequency=f), c=microstructure_colour_list()[microstructure], 
               label=microstructure_short_labels()[microstructure], alpha=alf, edgecolors='none', 
                 marker=microstructure_symbols()[microstructure])  for f in obs_tb.frequency] 
    [axh.scatter(obs_tb.TBAH.sel(frequency=f), res.TbH(snowpack=subset_snowpacks('B_abs'), frequency=f), c=microstructure_colour_list()[microstructure], 
               alpha=alf, edgecolors='none', marker=microstructure_symbols()[microstructure]) for f in obs_tb.frequency]
    [axh.scatter(obs_tb.TBMH.sel(frequency=f), res.TbH(snowpack=subset_snowpacks('A_met'), frequency=f), c=microstructure_colour_list()[microstructure],
               label=microstructure_short_labels()[microstructure], alpha=alf, edgecolors='none', 
                 marker=microstructure_symbols()[microstructure])  for f in obs_tb.frequency] 
    [axh.scatter(obs_tb.TBMH.sel(frequency=f), res.TbH(snowpack=subset_snowpacks('B_met'), frequency=f), c=microstructure_colour_list()[microstructure], 
               alpha=alf, edgecolors='none', marker=microstructure_symbols()[microstructure]) for f in obs_tb.frequency]

# Used for calculation of RMSE and ME    
def format_obs_for_error_statistics(tb, pol):
    # Alternates absorber / metal tb for each slab sample 
    # Allows direct comparison (statistics calculations) with SMRT results
    
    # Reference H or V pol
    if pol == 'V':
        tb_abs = tb.TBAV
        tb_met = tb.TBMV
    elif pol == 'H':
        tb_abs = tb.TBAH
        tb_met = tb.TBMH
    else:
        print ('pol not recognized')
        return
    
    # Make a list of 1-D xarray data arrays, then concatenate them
    all_obs = []
    for f in tb_abs.frequency:
        # Need two sets for two micro-CT samples from same slab
        obs_list = [xr.concat([tb_abs.sel(slab=s, frequency=f), tb_met.sel(slab=s, frequency=f),
          tb_abs.sel(slab=s, frequency=f), tb_met.sel(slab=s, frequency=f)], dim='slab') for s in obs_tb.TBAH.slab]
        all_obs.append(xr.concat(obs_list, dim='slab'))
    # Make coords consistent with results
    return xr.concat(all_obs, dim='frequency').rename({'slab':'snowpack'}).assign_coords(snowpack=results_exp.TbV().snowpack.values)

#### Set up of emissivity dictionary keys

In [ ]:
# Make tuple key dictionary
# List of tuple keys - this will stay the same regardless of slab and only needs to be defined once.
emiss_keys = []
for f in rads.frequency:
    emiss_keys.append((f, 'H'))
    emiss_keys.append((f, 'V'))

#### Make list of snowpacks for each microstructure model

In [ ]:
# Get list of samples i.e.. A and B for each slab
sample_list = sorted([s + 'A' for s in slab_list] + [s + 'B' for s in slab_list])

# Get list of snowpacks - will have one list per microstructure model
snowpack_list = sorted([s + '_abs' for s in sample_list] + [s + '_met' for s in sample_list])

In [ ]:
exp_snowpacks = [asmex_snowpack(s, 'Exponential') for s in snowpack_list]
shs_snowpacks = [asmex_snowpack(s, 'StickyHardSpheres') for s in snowpack_list]
ind_snowpacks = [asmex_snowpack(s, 'IndependentSphere') for s in snowpack_list]
ts_snowpacks = [asmex_snowpack(s, 'TeubnerStrey') for s in snowpack_list]
grf_snowpacks = [asmex_snowpack(s, 'GaussianRandomField') for s in snowpack_list]

#### Run model for each microstructure

In [ ]:
# Run model on exponential packs
results_exp = model.run(rads, exp_snowpacks, snowpack_dimension=('snowpack', snowpack_list))

In [ ]:
# Run model on independent sphere packs
results_ind = model.run(rads, ind_snowpacks, snowpack_dimension=('snowpack', snowpack_list))

In [ ]:
# Run model on sticky hard sphere packs
results_shs = model.run(rads, shs_snowpacks, snowpack_dimension=('snowpack', snowpack_list))

In [ ]:
# Run model on teubner strey packs
results_ts = model.run(rads, ts_snowpacks, snowpack_dimension=('snowpack', snowpack_list))

In [ ]:
# Run model on gaussian random field packs
results_grf = model.run(rads, grf_snowpacks, snowpack_dimension=('snowpack', snowpack_list))

#### Plot results

In [ ]:
plt.close()
fig, (axv, axh) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(6,12))
plt.rcParams.update({'font.size': 16})

plot_tb('Exponential')
plot_tb('IndependentSphere')
plot_tb('StickyHardSpheres')
plot_tb('TeubnerStrey')
plot_tb('GaussianRandomField')

#1-1 line
x = np.linspace(0, 280, 10)
for ax in [axv, axh]:
    ax.plot(x,x,'k--', alpha=0.3)
    ax.set_xlim([0, 275])
    ax.set_ylim([0, 275])
    ax.set_ylabel('Simulated TB (K)')

plt.xlabel('Observed TB (K)')
plt.tight_layout()
plt.show()
fig.savefig('Fig6a.png')

#### Calculate error statistics

In [ ]:
HobsTB = format_obs_for_error_statistics(obs_tb, 'H')
VobsTB = format_obs_for_error_statistics(obs_tb, 'V')

In [ ]:
# For paper. RMSE (H, V)
print ('\nRoot Mean Squared Error (H, V)')
print ('EXP', np.round(rmse(HobsTB, results_exp.TbH()),1),
      np.round(rmse(VobsTB, results_exp.TbV()),1))
print ('SHS', np.round(rmse(HobsTB, results_shs.TbH()),1),
      np.round(rmse(VobsTB, results_shs.TbV()),1))
print ('IND', np.round(rmse(HobsTB, results_ind.TbH()),1),
      np.round(rmse(VobsTB, results_ind.TbV()),1))
print ('TS', np.round(rmse(HobsTB, results_ts.TbH()),1),
      np.round(rmse(VobsTB, results_ts.TbV()),1))
print ('GRF', np.round(rmse(HobsTB, results_grf.TbH()),1),
      np.round(rmse(VobsTB, results_grf.TbV()),1))

In [ ]:
# For paper. ME (H, V)
print ('\nMean Error (H, V)')
print ('EXP', np.round(me(HobsTB, results_exp.TbH()),1),
      np.round(me(VobsTB, results_exp.TbV()),1))
print ('SHS', np.round(me(HobsTB, results_shs.TbH()),1),
      np.round(me(VobsTB, results_shs.TbV()),1))
print ('IND', np.round(me(HobsTB, results_ind.TbH()),1),
      np.round(me(VobsTB, results_ind.TbV()),1))
print ('TS', np.round(me(HobsTB, results_ts.TbH()),1),
      np.round(me(VobsTB, results_ts.TbV()),1))
print ('GRF', np.round(me(HobsTB, results_grf.TbH()),1),
      np.round(me(VobsTB, results_grf.TbV()),1))

In [ ]:
# Determine whether SHS or TS has smallest ME in Hpol
print (me(HobsTB, results_shs.TbH()), me(HobsTB, results_ts.TbH()))